전이학습
  - 파이썬을 배운사람은 다른언어도 좀더 쉽게 배울수 있음
  - 영상관련해서는 데이터를 확보는게 어렵다.. 기존 방대한 데이터로 학습된 모델을 가져와서 이용

ImageNet으로 학습된 ResNet50을 cub 데이터셋으로 전이학습

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from tensorflow.keras.preprocessing import image
import os

In [ ]:
!unzip "/content/drive/MyDrive/data/cub200.zip"

In [3]:
from glob import glob

In [4]:
file_lists = glob("/content/CUB_200_2011/images/**/*.jpg")
file_lists[:3]

['/content/CUB_200_2011/images/145.Elegant_Tern/Elegant_Tern_0005_150708.jpg',
 '/content/CUB_200_2011/images/145.Elegant_Tern/Elegant_Tern_0084_150922.jpg',
 '/content/CUB_200_2011/images/145.Elegant_Tern/Elegant_Tern_0066_150864.jpg']

In [20]:
class_reduce = 0.1
no_count = int(len([ fpath.split('/')[4] for fpath in file_lists])*class_reduce)

In [6]:
x_train = []; y_train=[]
train_file_paths = file_lists[:no_count]
temp = np.unique([ i.split('/')[4] for i in train_file_paths])
ylabel_dict = dict(zip( temp, list(range(len(temp))) ))
ylabel_dict

{'003.Sooty_Albatross': 0,
 '013.Bobolink': 1,
 '021.Eastern_Towhee': 2,
 '030.Fish_Crow': 3,
 '052.Pied_billed_Grebe': 4,
 '065.Slaty_backed_Gull': 5,
 '067.Anna_Hummingbird': 6,
 '097.Orchard_Oriole': 7,
 '100.Brown_Pelican': 8,
 '116.Chipping_Sparrow': 9,
 '119.Field_Sparrow': 10,
 '128.Seaside_Sparrow': 11,
 '145.Elegant_Tern': 12,
 '154.Red_eyed_Vireo': 13,
 '170.Mourning_Warbler': 14,
 '174.Palm_Warbler': 15,
 '175.Pine_Warbler': 16,
 '179.Tennessee_Warbler': 17,
 '181.Worm_eating_Warbler': 18,
 '188.Pileated_Woodpecker': 19}

In [30]:
x_tests = file_lists[no_count:][:5]  # 119.Field_Sparrow    10
x_test,y_test = [],[]
for xt in x_tests:
  img = image.load_img( xt ,target_size=(224,224))
  if len(img.getbands()) != 3:
    print(f'유효하지 않은 영상 {xt}')
    continue
  x = image.img_to_array(img)
  x = preprocess_input(x)  # 전이학습 모델의 입력형태로 변경
  x_test.append(x)  # 학습용 데이터
  y_test.append( ylabel_dict[ xt.split('/')[4] ] )
x_test = np.array(x_test); y_test = np.array(y_test)
x_test.shape, y_test.shape

((5, 224, 224, 3), (5,))

In [31]:
y_test

array([10, 10, 10, 10, 10])

In [7]:
x_train,y_train = [],[]
for img_file_path in train_file_paths:
  img = image.load_img( img_file_path ,target_size=(224,224))
  if len(img.getbands()) != 3:
    print(f'유효하지 않은 영상 {img_file_path}')
    continue
  x = image.img_to_array(img)
  x = preprocess_input(x)  # 전이학습 모델의 입력형태로 변경
  x_train.append(x)  # 학습용 데이터
  y_train.append( ylabel_dict[ img_file_path.split('/')[4] ] )
x_train = np.array(x_train)
y_train = np.array(y_train)

In [8]:
x_train.shape, y_train.shape, y_train[:5]

((1179, 224, 224, 3), (1179,), array([12, 12, 12, 12, 12]))

In [9]:
no_class = len(np.unique(y_train))
y_train_one_hot = tf.keras.utils.to_categorical(y_train,no_class)


In [10]:
base_model = ResNet50(include_top=False,input_shape=(224, 224,3))
cnn =Sequential()
cnn.add(base_model)
cnn.add(Flatten())
cnn.add(Dense(1024,activation='relu'))
cnn.add(Dense(no_class,activation='softmax'))

94765736/94765736 [==============================] - 3s 0us/step


In [ ]:
base_model.summary()

In [ ]:
cnn.compile(loss='categorical_crossentropy', optimizer=Adam(),metrics=['accuracy'])
hist = cnn.fit(x_train,y_train_one_hot,epochs = 10)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(hist.history['loss'])
plt.show()

In [25]:
np.unique(y_train)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [35]:
x_train.shape, y_train_one_hot.shape, x_test.shape, y_test.shape
# cnn.evaluate(x_test,y_test)

((1179, 224, 224, 3), (1179, 20), (5, 224, 224, 3), (5,))

이미지 제너레이터 사용

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_data_dir = '/content/CUB_200_2011/images'
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
    )
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'categorical',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'categorical',
    subset='validation'
)

Found 9472 images belonging to 200 classes.
Found 2324 images belonging to 200 classes.


In [3]:
no_class = 200
base_model = ResNet50(include_top=False,input_shape=(224, 224,3))
cnn =Sequential()
cnn.add(base_model)
cnn.add(Flatten())
cnn.add(Dense(1024,activation='relu'))
cnn.add(Dense(no_class,activation='softmax'))

cnn.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(),metrics=['accuracy'] )

In [4]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 1024)              102761472 
                                                                 
 dense_1 (Dense)             (None, 200)               205000    
                                                                 
Total params: 126554184 (482.77 MB)
Trainable params: 126501064 (482.56 MB)
Non-trainable params: 53120 (207.50 KB)
_________________________________________________________________


In [ ]:
hist = cnn.fit(
    train_generator,
    validation_data = validation_generator,
    epochs = 10
)

Epoch 1/10
